# Packed Ensemble Application to the AirfRANS dataset

## Colab setup

Set `colab` to `True` if you wish to use it 

In [ ]:
colab = False
if colab:
    from google.colab import drive

    drive.mount("/content/drive")
    !source/content/drive/MyDrive/my_colab_env/bin/activate
    import sys
    import os

    sys.path.append("/content/drive/MyDrive/my_colab_env/lib/python3.10/site-packages")
    os.chdir("/content/drive/MyDrive/ml4science/ml4physim_startingkit")

    sys.path.append(os.getcwd())

## Generic Step (Load the required data) <a id='generic_step'></a>

Install the LIPS framework if it is not already done. For more information look at the LIPS framework [Github repository](https://github.com/IRT-SystemX/LIPS)

In [ ]:
# !pip install -r requirements.txt
# or
# !pip install -U .


Install the AirfRANS package

In [ ]:
# !pip install airfrans

## Generic Step (Load the required data) <a id='generic_step'></a>

In [ ]:
import math
import os
from lips import get_root_path

In [ ]:
# indicate required paths
LIPS_PATH = get_root_path()
DIRECTORY_NAME = '../ml4physim_startingkit/Dataset'
BENCHMARK_NAME = "Case1"
LOG_PATH = LIPS_PATH + "lips_logs.log"

Define the configuration files path, that aim to describe specific caracteristics of the use case or the augmented simulator.

In [ ]:
BENCH_CONFIG_PATH = os.path.join("airfoilConfigurations", "benchmarks",
                                 "confAirfoil.ini")  #Configuration file related to the benchmark
SIM_CONFIG_PATH = os.path.join("airfoilConfigurations", "simulators", "torch_fc.ini")  #Configuration file re

Download the data

In [ ]:
not os.path.isdir(DIRECTORY_NAME)

In [ ]:
from lips.dataset.airfransDataSet import download_data

if not os.path.isdir(DIRECTORY_NAME):
    download_data(root_path=".", directory_name=DIRECTORY_NAME)

Loading the dataset using the dedicated class used by LIPS platform offers a list of advantages:

1. Ease the importing of datasets
1. A set of functions to organize the `inputs` and `outputs` required by augmented simulators


In [ ]:
# Load the required benchmark datasets
from lips.benchmark.airfransBenchmark import AirfRANSBenchmark
import pickle

try:
    with open('benchmark.pkl', 'rb') as f:
        benchmark = pickle.load(f)
except:
    benchmark = AirfRANSBenchmark(benchmark_path=DIRECTORY_NAME,
                                  config_path=BENCH_CONFIG_PATH,
                                  benchmark_name=BENCHMARK_NAME,
                                  log_path=LOG_PATH)
    benchmark.load(path=DIRECTORY_NAME)
    with open('benchmark.pkl', 'wb') as f:
        pickle.dump(benchmark, f)

## Model selection (Cross validation)

Importing the necessary dependencies, as well as the `packed_ensemble` methods

In [ ]:
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
import itertools as it
from typing import Union
import matplotlib.pyplot as plt
from lips.dataset.scaler.standard_scaler import StandardScaler
from my_packed_ensemble import *

In [ ]:
def build_k_indices(num_row, k_fold, seed):
    """build k indices for k-fold.

    Parameters
    ----------
    num_row : int
        Number of rows in the dataset.
    k_fold : int
        Number of folds
    seed : int
        Seed for random generator

    Returns
    -------
    k_indices : np.array
        Array of indices for each fold"""
    interval = int(num_row / k_fold)
    np.random.seed(seed)
    indices = np.random.permutation(num_row)
    k_indices = [indices[k * interval: (k + 1) * interval] for k in range(k_fold)]
    return np.array(k_indices)

In [ ]:
def save_losses(train_losses_list: list, val_losses_list: list, folder: str, file_name: str):
    """
    Saves the training and validation losses.

    Parameters
    ----------
    train_losses_list : list
        List containing the training losses.
    val_losses_list : list
        List containing the validation losses.
    folder : str
        Folder where the losses will be saved.
    file_name : str
        Name of the file where the losses will be saved.
    """

    # create folder if it does not exist
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # save losses
    df = pd.DataFrame({'train_loss': train_losses_list, 'val_loss': val_losses_list})
    df.to_csv(folder + "/" + file_name, index=False)


In [ ]:
def save_training_validation_losses_plot(train_losses_list: list, val_losses_list: list,
                                         hyperparam_dict: dict, folder: str, plot_name: str):
    """
    Saves the training and validation losses plot.

    Parameters
    ----------
    train_losses_list : list
        List containing the training losses.
    val_losses_list : list
        List containing the validation losses.
    """

    # create folder if it does not exist
    if not os.path.isdir(folder):
        os.makedirs(folder)

    # clear previous plot
    plt.clf()

    plt.plot(train_losses_list, label='Training loss', color='blue')
    plt.plot(val_losses_list, label='Validation loss', color='red')

    plt.ylabel('Loss')
    plt.xlabel('Epoch')

    plt.title(
        f'Losses for hidden_sizes={hyperparam_dict["hidden_sizes"]}, dropout={hyperparam_dict["dropout"]}, M={hyperparam_dict["M"]}, \n alpha={hyperparam_dict["alpha"]}, gamma={hyperparam_dict["gamma"]}, lr={hyperparam_dict["lr"]}')
    plt.legend()
    plt.savefig(folder + "/" + plot_name)

Create cross validation on hyperparameters of the model

In [ ]:
def get_hyper_dict(param_grid: dict):
    """
    Returns a grid of hyperparameters.

    Parameters
    ----------
    param_grid : dict
        Dictionary containing the hyperparameters.

    Returns
    -------
    hyperparameter_dict : dict
        Dictionary containing the list of hyperparameters with the ith element of each list being one unique combination of the hyperparameters.
    """

    # generate all combinations of parameter values
    combinations = it.product(*(param_grid[key] for key in param_grid))

    # create a new dictionary with keys as hyperparameter names and values as lists of combinations
    hyperparameter_dict = {key: [] for key in param_grid}

    # fill in the values for each key in the new dictionary
    for combo in combinations:
        for i, key in enumerate(param_grid):
            hyperparameter_dict[key].append(combo[i])
    
    return hyperparameter_dict

def sample_data(data_x, data_y, size_scale, seed:Union[int, None] = None):
    """
    Samples the data to reduce the size of the dataset to size_scale.

    Parameters
    ----------
    data_x : np.array
        Array containing the input data.
    data_y : np.array
        Array containing the output data.
    size_scale : float
        Percentage of the data to be sampled.

    Returns
    -------
    processed_x : np.array
        Array containing the sampled input data.
    processed_y : np.array
        Array containing the sampled output data.        
    """

    if seed is not None:
        np.random.seed(seed)

    sample_indices = np.arange(data_x.shape[0])
    np.random.shuffle(sample_indices)
    sample_indices = sample_indices[:int(size_scale*data_x.shape[0])]

    processed_x = data_x[sample_indices]
    processed_y = data_y[sample_indices]

    return processed_x, processed_y

In [ ]:
def hyperparameters_tuning(benchmark: DataSet, param_grid: dict, k_folds: int, num_epochs: int, batch_size: int = 128000,
                           shuffle: bool = False, n_workers: int = 0, seed: int = 42, scaler: Scaler = None,
                           partition: int = 0, verbose=False, size_scale=0.3, device="cpu"):
    """
    Performs hyperparameter tuning using K-fold cross validation.

    Parameters
    ----------
    param_grid : dict
        Dictionary containing the values for each hyperparameter to be tested.
    k_folds : int
        Number of folds to be used in the cross validation.
    num_epochs : int
        Number of epochs to be used in the training.
    batch_size : int
        Batch size to be used in the training.
    shuffle : bool
        Whether to shuffle the training dataset.
    n_workers : int
        Number of workers to be used in the training.
    seed : int
        Random seed to be used in the training.
    scaler : Scaler
        Scaler to be used in the model.
    partition : int
        Partition of the hyperparameter grid to be used in this run.

    Returns
    -------
    0 : int
        Returns 0 if the function runs successfully.
    """

    # set the random seed
    torch.manual_seed(seed)
    np.random.seed(seed)

    if verbose: print(f"Device: {device}")

    # get the hyperparameter grid dictionary
    hyperparameter_dict = get_hyper_dict(param_grid)
    hyperparameters_size = len(hyperparameter_dict[list(hyperparameter_dict.keys())[0]])

    dataset = benchmark.train_dataset
    input_size, output_size = infer_input_output_size(dataset)

    extract_x, extract_y = dataset.extract_data()

    # sample only a fraction of the dataset
    extract_x, extract_y = sample_data(extract_x, extract_y, size_scale, seed=None)

    results_df = pd.DataFrame(columns=[*param_grid.keys(), "validation_loss"])

    try:
        # read the checkpoint
        with open(f"CV/partition_{partition}/checkpoint_{partition}.txt", "rb") as f:
            checkpoint = int(f.read())
    except:
        checkpoint = -1

    indices = range(hyperparameters_size)

    if partition == 0:
        indices = indices[:hyperparameters_size // 3]
    elif partition == 1:
        indices = indices[hyperparameters_size // 3: 2 * hyperparameters_size // 3]
    elif partition == 2:
        indices = indices[partition * hyperparameters_size // 3:]
    else:
        raise ValueError("Invalid partition value. It must be 0, 1 or 2.")

    # remove the indices that have already been processed
    indices = indices[checkpoint + 1:]
    if verbose: indices = tqdm(indices)
    for i in indices:
        param_dict = {
            'hidden_sizes': hyperparameter_dict["hidden_sizes"][i],
            'dropout': hyperparameter_dict["dropout"][i],
            'M': hyperparameter_dict["M"][i],
            'alpha': hyperparameter_dict["alpha"][i],
            'gamma': hyperparameter_dict["gamma"][i],
            'lr': hyperparameter_dict["lr"][i]
        }

        if verbose:
            print(f'Hyperparameters: {i}/hidden_sizes={hyperparameter_dict["hidden_sizes"][i]}, \
                    dropout={hyperparameter_dict["dropout"][i]}, M={hyperparameter_dict["M"][i]}, alpha={hyperparameter_dict["alpha"][i]}, \
                    gamma={hyperparameter_dict["gamma"][i]}, lr={hyperparameter_dict["lr"][i]}')

        # define the K-fold Cross Validator
        k_indices = build_k_indices(extract_y.shape[0], k_folds, seed=seed)
        summed_validation_loss = 0

        # k-fold Cross Validation model evaluation
        for fold in range(k_folds):
            if verbose: print(f"fold: {fold}")

            # initialize the Packed MLP model
            model = PackedMLP(
                input_size=input_size,
                output_size=output_size,
                hidden_sizes=hyperparameter_dict["hidden_sizes"][i],
                activation=F.relu,
                device=device,
                dropout=hyperparameter_dict["dropout"][i],
                M=hyperparameter_dict["M"][i],
                alpha=hyperparameter_dict["alpha"][i],
                gamma=hyperparameter_dict["gamma"][i],
                scaler=scaler
            )
            model.to(device)

            val_ids = k_indices[fold]
            train_ids = k_indices[~(np.arange(k_indices.shape[0]) == fold)]

            train_x = extract_x[train_ids]
            train_y = extract_y[train_ids]

            train_x = train_x.reshape(train_x.shape[0] * train_x.shape[1], -1)
            train_y = train_y.reshape(train_y.shape[0] * train_y.shape[1], -1)

            val_x = extract_x[val_ids]
            val_y = extract_y[val_ids]

            trainloader = model.process_dataset(data=(train_x, train_y), training=True, batch_size=batch_size,
                                                shuffle=shuffle, n_workers=n_workers)
            validateloader = model.process_dataset(data=(val_x, val_y), training=False, batch_size=batch_size,
                                                   shuffle=shuffle, n_workers=n_workers)

            model, train_losses, val_losses = train(model=model, train_loader=trainloader, val_loader=validateloader,
                                                    epochs=num_epochs, device=device, lr=hyperparameter_dict["lr"][i],
                                                    verbose=verbose)

            summed_validation_loss += np.mean(val_losses)

            # saving the losses
            save_losses(train_losses_list=train_losses, val_losses_list=val_losses,
                        folder=f"CV/partition_{partition}/losses_{partition}/hyperparameters_{i}", file_name=f'fold_{fold}.csv')

            # saving the curve
            save_training_validation_losses_plot(train_losses_list=train_losses, val_losses_list=val_losses,
                                                 hyperparam_dict=param_dict, folder=f"CV/partition_{partition}/plots_{partition}/hyperparameters_{i}",
                                                 plot_name=f'fold_{fold}.png')

        mean_validation_loss = summed_validation_loss / k_folds
        
        if verbose:
            # print fold results
            print(f'FOLD {fold} RESULTS FOR {i}th HYPERPARAMETERS')
            print(f'Average validation loss: {mean_validation_loss}')
            print('-------------------------------- \n\n')

        param_dict.update({'validation_loss': mean_validation_loss})
        results_df.loc[len(results_df)] = param_dict
        
        # save the results
        try:
            pd.read_csv(f"CV/partition_{partition}/results_{partition}.csv").append(results_df).to_csv(f"CV/partition_{partition}/results.csv", index=False)
        except:
            results_df.to_csv(f"CV/partition_{partition}/results_{partition}.csv", index=False)

        checkpoint += 1
        # save the checkpoint
        with open(f"CV/partition_{partition}/checkpoint_{partition}.txt", "wb") as f:
            f.write(str(checkpoint).encode())

    return 0

In [ ]:
param_grid = {
    'hidden_sizes': [(48, 128, 48), (128, 256, 128)],
    'dropout': [True, False],
    "alpha": [2, 4],
    "gamma": [2, 4],
    "M": [4],
    'lr': [1e-2, 1e-3]
}

In [ ]:
# param_grid = {
#     'hidden_sizes': [(48, 128, 48)],
#     'dropout': [True],
#     "alpha": [2],
#     "gamma": [1],
#     "M": [4],
#     'lr': [3e-4],
# }

The `param_grid` will be divided in 3 partitions, each one will be executed on a different machine.

- Anton - partition 0
- Anthony - partition 1
- Alexi - partition 2

Change it in the cell below.

In [ ]:
partition = 0
device="cuda" if torch.cuda.is_available() else "cpu"

# hyperparameter tuning using CV
hyperparameters_tuning(benchmark=benchmark, param_grid=param_grid, k_folds=4, num_epochs=1, batch_size=1280000, shuffle=True, n_workers=6,
                        scaler=StandardScaler(), partition=partition, verbose=True, size_scale=0.3, device=device)

## Model training

In [ ]:
input_size, output_size = infer_input_output_size(benchmark.train_dataset)

In [ ]:
# device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = PackedMLP(input_size=input_size,
                  output_size=output_size,
                  hidden_sizes=(50, 100, 50),
                  activation=F.relu,
                  device=device,
                  dropout=True,
                  )
model.to(device)
model.device

In [ ]:
train_loader = model.process_dataset(benchmark.train_dataset, training=True, n_workers=6)

In [ ]:
print(model)

In [ ]:
model, train_losses, _ = train(model, train_loader, epochs=1, device=device, lr=3e-4)

##### prediction on `test_dataset`
This dataset has the same distribution as the training set

In [ ]:
predictions, observations = predict(model, benchmark._test_dataset, device=device)

In [ ]:
print("Prediction dimensions: ", predictions["x-velocity"].shape, predictions["y-velocity"].shape,
      predictions["pressure"].shape, predictions["turbulent_viscosity"].shape)
print("Observation dimensions:", observations["x-velocity"].shape, observations["y-velocity"].shape,
      observations["pressure"].shape, observations["turbulent_viscosity"].shape)
print("We have good dimensions!")

In [ ]:
from lips.evaluation.airfrans_evaluation import AirfRANSEvaluation

evaluator = AirfRANSEvaluation(config_path=BENCH_CONFIG_PATH,
                               scenario=BENCHMARK_NAME,
                               data_path=DIRECTORY_NAME,
                               log_path=LOG_PATH)

observation_metadata = benchmark._test_dataset.extra_data
metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)

##### Prediction on `test_ood_dataset`
This dataset has a different distribution in comparison to the training set.

In [ ]:
predictions, observations = predict(model, benchmark._test_ood_dataset, device=device)
evaluator = AirfRANSEvaluation(config_path=BENCH_CONFIG_PATH,
                               scenario=BENCHMARK_NAME,
                               data_path=DIRECTORY_NAME,
                               log_path=LOG_PATH)

metrics = evaluator.evaluate(observations=observations,
                             predictions=predictions,
                             observation_metadata=observation_metadata)
print(metrics)